In [1]:
import csv
import numpy as np 
import pandas as pd
from datetime import datetime as dt
import pickle

pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', 200)

**Get all user IDs**

all user ids, stored in 'userID_dict' in the format of:

userID_dict[user_id] = username

In [2]:
# temp1D = pd.read_csv("dict_1d_usernames.csv", low_memory=False)
temp1D = pd.read_csv("/home/updu6059/Twint_Data/ParallelProcessing/Famous_Users/DidNotMentionBack_Users/Filtered_EnglishUsers_DidNotMentionedBackUsers_FINAL.csv", low_memory=False)
# temp1D.head()
temp1D.head()
len(temp1D)

3102

In [3]:
import tweepy
# consumer_key = 'nnqF5rgtW0CKLjr2r4wLMdqMr'
# consumer_secret = 'Ny2LDN43ZB5WCGMkKpLwc2FXbFN9x43UwYeQzACmuA4W4zKsLS'
# access_token = '722619569760641025-mFPOJmQtCBwCg9d9LeQGUwFDdrPQCtE'
# access_token_secret = 'Xi4h0a1WpcmrKjfxtNTXYTvzximuF9FzzKDJOptPgOS7X'
consumer_key = 'bVgAX5fGidoy2IgPKHHgcfOG4'
consumer_secret = '83b6zN5HSIjPk7ALaGwcFY9DRwNEvOABgB0YD3TJI4KXigzJY9'
access_token = '48478289-5oNed2t3pcbO5aSt5DdeoeoOTfLUePQUYJ96tkkfd'
access_token_secret = 'VDOFj9IjC0r5QgbIu1kGgAp68fEwHjKLlhGJkBPFRNbXD'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
# user = api.get_user(screen_name = 'notnotrhett')
# user.id

In [4]:
userID_dict = {}
import time
# for i in range(len(temp1D)):
#     userID_dict[temp1D['user_id'][i]] = temp1D['username'][i]

for i in range(len(temp1D)):
    try:
        user = api.get_user(screen_name = temp1D['Users'][i])
        userID_dict[user.id] = temp1D['Users'][i]
#         print(user.id)
    except tweepy.RateLimitError:
        print("RATE LIMIT HIT [" + str(i) + "]- " + temp1D['Users'][i])
        time.sleep(15 * 60)
        i -= 1
    except:
        print("User Failed [" + str(i) + "]- " + temp1D['Users'][i])
    
len(userID_dict)

User Failed [228]- SecludedSusie
User Failed [480]- pzf
User Failed [671]- TulipLotus
User Failed [689]- DocPakistan
User Failed [777]- thetrudz
RATE LIMIT HIT [900]- eleanorbate
User Failed [950]- DDonsmith007
User Failed [1002]- MsBlaireWhite
User Failed [1031]- BratFrye
User Failed [1096]- OfficialBrettS
User Failed [1223]- Nancys_News
User Failed [1645]- pzf
RATE LIMIT HIT [1801]- arielhelwani
User Failed [2253]- GodlessNZ
RATE LIMIT HIT [2702]- CRRJA5
User Failed [3066]- slumpcone26


2918

**Read in IRA tweet set**

In [ ]:
iraDF = pd.read_csv('ira_tweets_csv_unhashed.csv', low_memory=False)

In [ ]:
print(len(iraDF))

In [ ]:
iraDF.tail(5)

**Create DataFrame for IRA -> User Contact Points**

In [ ]:
contactPointsDF = pd.DataFrame(columns=['username','user_id','ira_username','ira_user_id',
                                       'tweet', 'tweet_date', 'meantions_list'])

**Loop through IRA data set, checking for contact with our current user set, if contact was established, add to contactPointsDF**

In [ ]:
total = str(len(iraDF))
count = 1
#loop through ira tweet DF
for tweet in range(len(iraDF)):
    mention_string = str(iraDF['user_mentions'][tweet])
    #if mention list exists, save and reformat as it is a string not a list
    if mention_string != "[]" and mention_string != 'nan':  
        mention_list = mention_string.split(', ')
        mention_list[0] = mention_list[0][1:]
        mention_list[-1] = mention_list[-1][:-1]
        for x in range(len(mention_list)):
            mention_list[x] = int(mention_list[x])
#         for j in range(len(mention_list)):
#             mention_list[j] = int(mention_list[j])

        #check against user ids in our userID_dict
        for mention in mention_list:
            temp_key = mention
            if temp_key in userID_dict.keys():
#                 print("found!")
                #if a user id was mentioned, add to contactPointsDF
                contactPointsDF = contactPointsDF.append(
                    {'username' : userID_dict[temp_key],
                      'user_id' : temp_key,
                      'ira_username' : iraDF['user_screen_name'][tweet],
                      'ira_user_id' : iraDF['userid'][tweet],
                      'tweet' : iraDF['tweet_text'][tweet],
                      'tweet_date' : iraDF['tweet_time'][tweet],
                      'meantions_list' : mention_list,
                     }, ignore_index=True)
    if count % 1000 == 0:
        print(str(count) + " / " + total)
    count += 1

**Now create a final DF which is more user-centric, i.e., a row for each user as opposed to a row for each tweet**


In [ ]:
contactedUsers = contactPointsDF.username.unique()

In [ ]:
len(contactPointsDF)

In [ ]:
contactPointsDF.to_csv('contact_IRAtweets_Inf_NONRESPONSIVE_v2.csv')

In [ ]:
len(contactedUsers)

In [ ]:
contactDF = pd.DataFrame(columns=['username','user_id','ira_username','ira_user_id',
                                       'tweet', 'tweet_date', 'mentions_list'])

In [ ]:
def get_key(val): 
    for key, value in userID_dict.items(): 
        if val == value: 
            return key

In [ ]:
print("hello")

In [ ]:
size = str(len(contactedUsers))
count = 1
for user in contactedUsers:
    ira_username_temp = []
    ira_userid_temp = []
    tweet_temp = []
    tweet_date_temp = []
    mentions_list_temp = []
    for tweet in range(len(contactPointsDF)):
        if(contactPointsDF['username'][tweet] == user):
            ira_username_temp.append(contactPointsDF['ira_username'][tweet])
            ira_userid_temp.append(contactPointsDF['ira_user_id'][tweet])
            tweet_temp.append(contactPointsDF['tweet'][tweet])
            tweet_date_temp.append(contactPointsDF['tweet_date'][tweet])
            mentions_list_temp.append(contactPointsDF['meantions_list'][tweet])
    contactDF = contactDF.append(
                    {'username' : user,
                      'user_id' : get_key(user),
                      'ira_username' : ira_username_temp,
                      'ira_user_id' : ira_userid_temp,
                      'tweet' : tweet_temp,
                      'tweet_date' : tweet_date_temp,
                      'mentions_list' : mentions_list_temp,
                     }, ignore_index=True)
    print(str(count) + " / " + size)
    count += 1

In [ ]:
contactDF.to_csv('contact_IRAtoUser_Inf_NONRESPONSIVE_v2.csv')

In [ ]:
print(len(contactDF))

In [ ]:
len(contactDF.username.unique())

In [ ]:
contactDF.head()

In [ ]:
# for i in range(len(contactDF)):
#     if contactDF['username'][i] == 'Borisovalustnah':
#         print(str(i))

In [ ]:
# print(str(len(contactDF) - 2265))

In [ ]:
newContactDF = contactDF

In [ ]:
len(contactDF)

In [ ]:
newContactDF.tail()

In [ ]:
newContactDF= newContactDF.reset_index()

In [ ]:
newContactDF = newContactDF.drop(['index'], axis=1)
newContactDF.head()

In [ ]:
# newContactDF.to_csv('contact_IRAtoUser.csv')

In [ ]:
# testing = newContactDF['mentions_list'][1]

In [ ]:
# len(testing)

In [ ]:
# tempdf = newContactDF
# tempdf['new'] = 0
# tempdf.head()

In [ ]:
newContactDF['contact_points'] = 0
# print(testing[0][0])
for h in range(len(newContactDF)):
# for h in range(100):
    testing = newContactDF['mentions_list'][h]
    temp = []
    for k in range(len(testing)):
        for l in range(len(testing[k])):
            temp.append(testing[k][l])
    newContactDF['mentions_list'][h] = temp
    newContactDF['contact_points'][h] = len(temp)
#     print(temp)

In [ ]:
newContactDF[100:200]

In [ ]:
len(newContactDF)

In [ ]:
# print(temp)

In [ ]:
newContactDF.to_csv('contact_IRAtoUser_Inf_NONRESPONSIVE_v2.csv')